# ERA5 + Merit Potential Runoff Climatology: Visualize

In [1]:
import ee
from statistics import mean
import earthshot as es
from earthshot import water_viz as vis 
from earthshot import water_common as common

Loading BokehJS ...

In [2]:
# ee.Authenticate()
ee.Initialize()

In [3]:
runoff_clim_path = 'users/jamesmcc/era5_merit_potential_sfc_runoff_conus_climatology'
runoff_clim = ee.ImageCollection(runoff_clim_path)

In [4]:
# The range of the data obscures the results of interest
# vis_range = vis.img_col_range(runoff_clim, area_of_interest=common.bboxes()['conus'])
# [0, 122472.83155243896]


In [5]:
# Beaver dams have flows in [.01, .5], so take something similar
vis_range = [0, .5]

In [6]:
vis_range

[0, 0.5]

In [7]:
palette_name = 'BrBG'
palette_len = 11
palette = vis.brewer[palette_name][palette_len][::-1]
vis.legend(palette=palette, minimum=vis_range[0], maximum=vis_range[1])

In [8]:
vis_params = {
    'min': vis_range[0], 'max': vis_range[1], 'dimensions': 512,
    'palette': palette}

## Static Image

In [9]:
variable = 'potential_sfc_runoff_mon_clim_cms'
mm = 1
img = (
    runoff_clim
    .filter(ee.Filter.eq('month', mm))
    .filter(ee.Filter.eq('variable', variable)).first())

In [10]:
display(vis.Image(url = img.getThumbURL(vis_params)))

## Folium Layers

In [11]:
box_corners = common.bboxes()['conus'].toGeoJSON()['coordinates'][0]
center_lon = mean([corner[0] for corner in box_corners])
center_lat = mean([corner[1] for corner in box_corners])

In [12]:
the_map = vis.folium_map(location=[center_lat, center_lon], zoom_start=4, height=500)
month_names = [
    'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
    'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

for mm0 in range(12):
    mm = mm0 + 1
    img = (
        runoff_clim
        .filter(ee.Filter.eq('month', mm))
        .filter(ee.Filter.eq('variable', variable)).first())
    the_map.add_ee_layer(img, vis_params, variable + ', ' + month_names[mm0])

vis.folium_display(the_map)

## geemap 

In [37]:
import geemap.eefolium as geemap

In [38]:
Map = geemap.Map()
landcover = ee.Image('MODIS/006/MCD12Q1/2013_01_01') \
    .select('LC_Type1') \

igbpLandCoverVis = {
  'min': 1.0,
  'max': 17.0,
  'palette': [
    '05450a', '086a10', '54a708', '78d203', '009900', 'c6b044', 'dcd159',
    'dade48', 'fbff13', 'b6ff05', '27ff87', 'c24f44', 'a5a5a5', 'ff6d4c',
    '69fff8', 'f9ffa4', '1c0dff'
  ],
}

Map.setCenter(6.746, 46.529, 2)
Map.addLayer(landcover, igbpLandCoverVis, 'MODIS Land Cover')
Map.add_legend(builtin_legend='MODIS/006/MCD12Q1')

Map.addLayerControl()

Map